# Task 1: Multi layer perceptron

In this task you will implement a multilayer perceptron which consists of multiple fully connected layers with biases each followed (potentially) by an activation function. This implementation is based on the python package numpy. Try to use numpy functions/matrices wherever possible.

We have already provided you with a function that initializes all the properties for you. The MLP is defined by providing the sizes of the layers (including input and output layers) and the activation functions that are applied after each layer. We will use vanilla stochastic gradient descent with mini-batches for weight & bias updates (but for simplicity we will only pass through one instance after another).

You can find implementations of activation functions below. They can be used in the forward pass but also provide the derivative if `deriv=True` is provided. See more details the comment in a).

Throughout the tasks do not change the arrays that are in the properties but rather use inplace changes if needed. You can e.g. do `self.gradients[0][:] += 1` to increase the value of all gradient entries of the first layer inplace.

### a) forward

Write a function `forward` that takes an input array that should then be passed through the entire network. All immediate layer results are stored in `y`. Thereby store also the input (raw input) and the output layer.
The forward pass goes as follows

$\mathbf{y_0} = \mathbf{x}$ <br>
for k=0.. do<br>
_    $z_{k} =  y_{k} W_k  + b_k$ <br>
_    $y_{k+1} = g_{k}(z_{k})$

Where $W$ is the weight matrix, $b$ is the bias vector and $g$ the activation function at layer $k$. Note that the $y_k$ introduced here are not the same as in the lecture they are more similar to the $u_k$ but still slightly different as we are not treating the activations as their own layer.

The variable $z_k$ is not stored but is here for illustration. In general one potentially would have to store it also but we don't need it for the activation functions sigmoid and relu. They have the special property that we can write those  as $g'(x) = G(g(x))$ for some function $G$.

### b) back propagation

The backpropagation is computed as:


$h \leftarrow \frac{\partial L}{\partial y}$<br>
for k = l, l-1,.. 1 do:<br>
_ $h \leftarrow \frac{\partial L}{\partial z_k} = h \odot g' _{k-1}(z_{k-1}) = h \odot G_{k-1}(y_k)$<br>
_ $\frac{\partial L}{\partial W_{k-1}} = y_{k-1} \otimes h$<br>
_ $\frac{\partial L}{\partial b_{k-1}} = h$<br>
_ $h \leftarrow \frac{\partial L}{\partial y_k} = h \cdot W_{k-1}$

Where $\odot$ is the element wise product, $\otimes$ is the outer product and $\cdot$ the dot product. Assume that a forward step has previosuly happened and has set the values of `results` accordingly. To keep it simple this function is intended for one instance only but we want to employ it in a mini batch scenario. So instead of directly overriding the gradients just add to them.

### tanh activation

Implement the tanh activation function such that they can also be used in your MLP. See e.g. https://en.wikipedia.org/wiki/Activation_function#Comparison_of_activation_functions


### d) update

Write a function update that updates the weights and biases for all layers based on a mini-batch (given as lists of input arrays `xs` and desired targets `ts` [$\hat y $ in the lecture]), loss function `loss_func` and and learning rate $\mu$. It therefore first zeros previous gradients (implement the `zero_grad` function), then does a forward and backwards pass for each training instance and finally updates the weights and biases using the learning rate.

This is a simplified process. In pratice all of the instances in the mini-batch would be passed forward and backwards simultanously we don't do it here.

### e) Evaluation functions

Implement the functions `calc_precision` and `calc_loss` that calculate the classification precision and the *average* loss over the provided examples `xs` and `ts`.


### f) Preapring real data
Write a function `top_n_words(sentences, n)` that returns a list of the most frequently used words for a corpus. You can assume that the function is tested such that there is no tie.

Then write a function `to_hot_encoding(sentences, top_words)` that encodes the document as a numpy array of type bool. It has the same length as `top_words` and indicates the presence or abscence of that particular word by 1 = present, 0 = absent. This is some kind of bag of words model.

The sentences are provided as a list of list of tokens. Do *not* do any cleaning on those tokens

### g) fiddle with hyper parameters

Find hyperparameters
```
my_seed = 1
my_sizes = (128, ...,2)
my_activations = (..., sigmoid)
my_epochs = 1
my_chunks = 1
```
such that you achieve >90% precision on the training set and >75% precision on the test set using the provided training  loop. You may only use `relu`, `identity`, `tanh` and `sigmoid` activation functions. The training time should not exceed 1min.

In [ ]:
import numpy as np

In [ ]:
class MLP:
    def __init__(self, sizes,activations):
        assert len(activations) == (len(sizes)-1)
        self.activation_functions = tuple(activations)


        # init weights, biases and temporary results
        self.weights = tuple(((np.random.random_sample((sizes[i], sizes[i+1]))-0.5) for i in range(len(sizes)-1)))
        self.biases = tuple((np.random.random_sample(sizes[i+1])-0.5) for i in range(len(sizes)-1))
        self.y = tuple(np.empty(sizes[i]) for i in range(len(sizes)))

        # init gradients
        self.gradients = tuple((np.zeros(arr.shape) for arr in self.weights))
        self.biases_gradients = tuple((np.zeros(arr.shape) for arr in self.biases))


    def forward(self, input_arr):
        # input_array is a numpy array,
        # this function returns nothing

        # set y0
        for i in range(len(input_arr)):
            self.y[0][i] = input_arr[i].astype(float)

        for layer in range(0, len(self.y)-1): # layer
            vals = self.activation_functions[layer](np.dot(self.y[layer], self.weights[layer]) + self.biases[layer])
            for o in range(len(vals)):
                self.y[layer+1][o] = vals[o]


    def back_prop(self, t, loss_func):
        # t : target label == desired form of the final label
        # loss_func: a loss function see squared loss below
        h = loss_func(self.y[len(self.y)-1], t, deriv=True)
        for k in range(len(self.y)-1, 0, -1):
            h =  h * self.activation_functions[k-1](self.y[k], deriv=True)
            self.gradients[k-1][:] += np.outer(self.y[k-1], h)
            self.biases_gradients[k-1][:] += h
            h = np.dot(self.weights[k-1], h)


    def zero_grad(self):
        for i in range(len(self.gradients)):
            self.gradients[i][:] = 0
            self.biases_gradients[i][:] = 0


    def update(self, xs, ts, loss_func, mu):
        # xs: list of numpy arrays (inputs)
        # ts: list of numpy arrays (desired output)
        # loss_func: function, a loss function see squared loss below
        # mu: float, learning rate
        #assert len(ts) == len(xs)
        self.zero_grad()
        for i in range(len(xs)):
            self.forward(xs[i])
            self.back_prop(ts[i], loss_func)
        for j in range(len(self.gradients)):
            self.weights[j][:] -= mu*self.gradients[j]
            self.biases[j][:] -= mu*self.biases_gradients[j]

    def calc_precision(self, xs, ts):
        # xs: list of numpy arrays (inputs)
        # ts: list of numpy arrays (desired output)
        right = 0
        for i in range(len(xs)):
            self.forward(xs[i])
            if (ts[i][self.y[len(self.y)-1].argmax()]):
                right += 1
        return right/len(xs)

    def calc_loss(self, xs, ts, loss_func):
        # xs: list of numpy arrays (inputs)
        # ts: list of numpy arrays (desired output)
        # loss_func: function, a loss function see squared loss below
        #arr = np.zeros(len(xs))
        loss = 0
        for i in range(len(xs)):
            self.forward(xs[i])
            #arr[i] = loss_func(self.y[len(self.y)-1], ts[i])
            loss += loss_func(self.y[len(self.y)-1], ts[i])
        return loss / len(xs)

In [ ]:
# activation functions
# if deriv=True they act the function capital G as explained above
def identity(x, deriv=False):
    if deriv == True:
        return 1
    return x

def relu(x, deriv=False):
    if deriv == True:
        out = np.zeros(x.shape)
        out[x>0]=x[x>0]
        return out
    return np.maximum(x, 0)

def sigmoid(x, deriv=False):
    if deriv == True:
        return x * (1 - x)
    return 1 / (1 + np.exp(-x))

def tanh(x, deriv=False):
    # print(tanh(np.array([1,2,3]))) [0.76159416 0.96402758 0.99505475]
    # print(tanh(np.array([1,2,3]), deriv=True)) [ 0 -3 -8]
    if deriv == True:
        return 1.0 - x**2
    return (np.exp(x) - np.exp(-x)) / (np.exp(x) + np.exp(-x))
    pass

In [ ]:
print(tanh(np.array([1,2,3])))
print(tanh(np.array([1,2,3]), deriv=True))

[0.76159416 0.96402758 0.99505475]
[ 0. -3. -8.]


In [ ]:
# loss function
def squared_loss(x, t, deriv=False):
    if deriv:
        return x - t
    return 0.5 * np.sum((np.square(x-t)))

In [ ]:
in1 = np.array([0,0,1,0,0], dtype=bool)
np.random.seed(1)

activs = [relu, relu, sigmoid]
myNN = MLP((5,4,3,2), activs)


myNN.forward(in1)
print(myNN.y[0])
print(myNN.y[1])
print(myNN.y[2])
print(myNN.y[-1])

#[0. 0. 1. 0. 0.]
#[0.         0.28896105 0.4080556  0.43338515]
#[0.         0.03587933 0.        ]
#[0.48869641 0.5991624 ]

[0. 0. 1. 0. 0.]
[0.         0.28896105 0.4080556  0.43338515]
[0.         0.03587933 0.        ]
[0.48869641 0.5991624 ]


In [ ]:
in2 = np.array([1,0,1,0,0], dtype=bool)
myNN.forward(in2)
print(myNN.y[0])
print(myNN.y[1])
print(myNN.y[2])
print(myNN.y[-1])
#[1. 0. 1. 0. 0.]
#[0.         0.50928554 0.         0.23571773]
#[0.         0.38629211 0.        ]
#[0.50550331 0.58354196]


[1. 0. 1. 0. 0.]
[0.         0.50928554 0.         0.23571773]
[0.         0.38629211 0.        ]
[0.50550331 0.58354196]


In [ ]:
myNN.zero_grad()
myNN.forward(in1)
myNN.back_prop(np.array([1,0]), squared_loss)

# untouched
print(myNN.weights)
print(myNN.biases)

# changed
print(myNN.gradients)
print(myNN.biases_gradients)

# (array([[-0.082978  ,  0.22032449, -0.49988563, -0.19766743],
#       [-0.35324411, -0.40766141, -0.31373979, -0.15443927],
#       [-0.10323253,  0.03881673, -0.08080549,  0.1852195 ],
#       [-0.29554775,  0.37811744, -0.47261241,  0.17046751],
#       [-0.0826952 ,  0.05868983, -0.35961306, -0.30189851]]),
#        array([[ 0.30074457,  0.46826158, -0.18657582],
#       [ 0.19232262,  0.37638915,  0.39460666],
#       [-0.41495579, -0.46094522, -0.33016958],
#       [ 0.3781425 , -0.40165317, -0.07889237]]),
#        array([[ 0.45788953,  0.03316528],
#       [ 0.19187711, -0.18448437],
#       [ 0.18650093,  0.33462567]]))
#(array([-0.48171172,  0.25014431,  0.48886109,  0.24816565]), array([-0.21955601,  0.28927933, -0.39677399]), array([-0.05210647,  0.4085955 ]))
#(array([[ 0.        ,  0.        ,  0.        ,  0.        ],
#       [ 0.        ,  0.        ,  0.        ,  0.        ],
#       [ 0.        , -0.00019926,  0.00034459,  0.00031891],
#       [ 0.        ,  0.        ,  0.        ,  0.        ],
#       [ 0.        ,  0.        ,  0.        ,  0.        ]]), array([[ 0.        ,  0.        ,  0.        ],
#       [ 0.        , -0.00052939,  0.        ],
#       [ 0.        , -0.00074758,  0.        ],
#       [ 0.        , -0.00079398,  0.        ]]), array([[ 0.        ,  0.        ],
#       [-0.00458396,  0.005163  ],
#       [ 0.        ,  0.        ]]))
#(array([ 0.        , -0.00019926,  0.00034459,  0.00031891]), array([ 0.        , -0.00183205,  0.        ]), array([-0.12776057,  0.14389893]))


(array([[-0.082978  ,  0.22032449, -0.49988563, -0.19766743],
       [-0.35324411, -0.40766141, -0.31373979, -0.15443927],
       [-0.10323253,  0.03881673, -0.08080549,  0.1852195 ],
       [-0.29554775,  0.37811744, -0.47261241,  0.17046751],
       [-0.0826952 ,  0.05868983, -0.35961306, -0.30189851]]), array([[ 0.30074457,  0.46826158, -0.18657582],
       [ 0.19232262,  0.37638915,  0.39460666],
       [-0.41495579, -0.46094522, -0.33016958],
       [ 0.3781425 , -0.40165317, -0.07889237]]), array([[ 0.45788953,  0.03316528],
       [ 0.19187711, -0.18448437],
       [ 0.18650093,  0.33462567]]))
(array([-0.48171172,  0.25014431,  0.48886109,  0.24816565]), array([-0.21955601,  0.28927933, -0.39677399]), array([-0.05210647,  0.4085955 ]))
(array([[ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.00019926,  0.00034459,  0.00031891],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
 

In [ ]:
# accumulates gradients over two examples
myNN.zero_grad()
myNN.forward(in2)
myNN.back_prop(np.array([1,0]), squared_loss)
myNN.forward(in2)
myNN.back_prop(np.array([1,0]), squared_loss)


# untouched
print(myNN.weights)
print(myNN.biases)

# changed
print(myNN.gradients)
print(myNN.biases_gradients)

#(array([[-0.082978  ,  0.22032449, -0.49988563, -0.19766743],
#       [-0.35324411, -0.40766141, -0.31373979, -0.15443927],
#       [-0.10323253,  0.03881673, -0.08080549,  0.1852195 ],
#       [-0.29554775,  0.37811744, -0.47261241,  0.17046751],
#       [-0.0826952 ,  0.05868983, -0.35961306, -0.30189851]]), array([[ 0.30074457,  0.46826158, -0.18657582],
#       [ 0.19232262,  0.37638915,  0.39460666],
#       [-0.41495579, -0.46094522, -0.33016958],
#       [ 0.3781425 , -0.40165317, -0.07889237]]), array([[ 0.45788953,  0.03316528],
#       [ 0.19187711, -0.18448437],
#       [ 0.18650093,  0.33462567]]))
#(array([-0.48171172,  0.25014431,  0.48886109,  0.24816565]), array([-0.21955601,  0.28927933, -0.39677399]), array([-0.05210647,  0.4085955 ]))
#(array([[ 0.        , -0.00738705,  0.        ,  0.00364851],
#       [ 0.        ,  0.        ,  0.        ,  0.        ],
#       [ 0.        , -0.00738705,  0.        ,  0.00364851],
#       [ 0.        ,  0.        ,  0.        ,  0.        ],
#       [ 0.        ,  0.        ,  0.        ,  0.        ]]), array([[ 0.        ,  0.        ,  0.        ],
#       [ 0.        , -0.01962609,  0.        ],
#       [ 0.        ,  0.        ,  0.        ],
#       [ 0.        , -0.00908374,  0.        ]]), array([[ 0.        ,  0.        ],
#       [-0.09549851,  0.10956233],
#       [ 0.        ,  0.        ]]))
#(array([ 0.        , -0.00738705,  0.        ,  0.00364851]), array([ 0.        , -0.03853652,  0.        ]), array([-0.24721839,  0.2836256 ]))


(array([[-0.082978  ,  0.22032449, -0.49988563, -0.19766743],
       [-0.35324411, -0.40766141, -0.31373979, -0.15443927],
       [-0.10323253,  0.03881673, -0.08080549,  0.1852195 ],
       [-0.29554775,  0.37811744, -0.47261241,  0.17046751],
       [-0.0826952 ,  0.05868983, -0.35961306, -0.30189851]]), array([[ 0.30074457,  0.46826158, -0.18657582],
       [ 0.19232262,  0.37638915,  0.39460666],
       [-0.41495579, -0.46094522, -0.33016958],
       [ 0.3781425 , -0.40165317, -0.07889237]]), array([[ 0.45788953,  0.03316528],
       [ 0.19187711, -0.18448437],
       [ 0.18650093,  0.33462567]]))
(array([-0.48171172,  0.25014431,  0.48886109,  0.24816565]), array([-0.21955601,  0.28927933, -0.39677399]), array([-0.05210647,  0.4085955 ]))
(array([[ 0.        , -0.00738705,  0.        ,  0.00364851],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.00738705,  0.        ,  0.00364851],
       [ 0.        ,  0.        ,  0.        ,  0.        ],
 

In [ ]:
# a function that generates some dummy training dataset
def generate_training(n):
    xs=[]
    ts=[]
    for _ in range(n):
        x = np.random.random_sample(10)>0.5
        t = (x[1] and x[2]) or x[3]
        xs.append(x)
        ts.append(np.array([t,~t]))
    return xs, ts
    return np.array(xs), np.array(ts)

In [ ]:
np.random.seed(1)
xs, ts = generate_training(1000)

In [ ]:
np.random.seed(1)
myNN = MLP((10,8,4, 2), activs)
print("#", myNN.calc_loss(xs,ts, squared_loss), myNN.calc_precision(xs, ts))

for x_chunk, t_chunk in zip(np.array_split(xs,100), np.array_split(ts,100)):
    myNN.update(x_chunk, t_chunk, squared_loss, 0.1)
    # evaluate NN on entire dataset
    print("#", myNN.calc_loss(xs,ts, squared_loss), myNN.calc_precision(xs, ts))

# 0.24257539789489851 0.634
# 0.2411544143560101 0.634
# 0.23984965535059738 0.634
# 0.23882087470347274 0.634
# 0.23843002829715368 0.634
# 0.2382914663918927 0.634
# 0.23764527725780307 0.634
# 0.23666737755900943 0.634
# 0.23620884986400037 0.634
# 0.2365034031129388 0.634
# 0.23616300372820928 0.634
# 0.23502874505528465 0.634
# 0.23503090772220395 0.634
# 0.23429557276477497 0.634
# 0.23407573863293182 0.634
# 0.2339776485620543 0.634
# 0.23348187532872453 0.634
# 0.23290833656298032 0.634
# 0.23273709472388385 0.634
# 0.2325444855821718 0.634
# 0.23267973317482651 0.634
# 0.23200642490995707 0.634
# 0.23170660058816217 0.634
# 0.2314284286110461 0.634
# 0.2310843786009857 0.634
# 0.2308711195328345 0.634
# 0.23047087788541018 0.634
# 0.23078563724882506 0.634
# 0.23010159343696798 0.634
# 0.22980399458119968 0.634
# 0.2297798677979091 0.634
# 0.2293841765404658 0.634
# 0.2291655512063339 0.634
# 0.22870691228278373 0.634
# 0.22817010393724937 0.634
# 0.22771610716266258 0.634
# 0

In [ ]:
# output from previous cell
# 0.24257539789489851 0.634
# 0.2411544143560101 0.634
# 0.23984965535059738 0.634
# 0.23882087470347274 0.634
# 0.23843002829715368 0.634
# 0.2382914663918927 0.634
# 0.23764527725780307 0.634
# 0.23666737755900943 0.634
# 0.23620884986400037 0.634
# 0.23650340311293885 0.634
# 0.23616300372820928 0.634
# 0.23502874505528465 0.634
# 0.23503090772220395 0.634
# 0.23429557276477497 0.634
# 0.23407573863293182 0.634
# 0.2339776485620543 0.634
# 0.23348187532872453 0.634
# 0.23290833656298032 0.634
# 0.23273709472388385 0.634
# 0.2325444855821718 0.634
# 0.23267973317482651 0.634
# 0.23200642490995707 0.634
# 0.23170660058816217 0.634
# 0.2314284286110461 0.634
# 0.2310843786009857 0.634
# 0.2308711195328345 0.634
# 0.23047087788541018 0.634
# 0.23078563724882506 0.634
# 0.23010159343696798 0.634
# 0.22980399458119968 0.634
# 0.2297798677979091 0.634
# 0.2293841765404658 0.634
# 0.2291655512063339 0.634
# 0.22870691228278373 0.634
# 0.22817010393724937 0.634
# 0.22771610716266258 0.634
# 0.22735622344120002 0.634
# 0.2268242701172851 0.634
# 0.22650100063083706 0.634
# 0.22550104232382934 0.634
# 0.22509370708150808 0.634
# 0.22467802344347212 0.634
# 0.2242107007448516 0.634
# 0.2233116482328667 0.634
# 0.22265497659346642 0.634
# 0.22178494678736085 0.634
# 0.2213379424960512 0.634
# 0.21990383266003793 0.634
# 0.21838496173099078 0.634
# 0.21753192292293697 0.634
# 0.21669299576563952 0.634
# 0.21663406404375596 0.634
# 0.2140600877981478 0.634
# 0.21325662957467928 0.634
# 0.21282023805393604 0.634
# 0.21274278318993375 0.634
# 0.21016631292648913 0.634
# 0.20932144343716885 0.634
# 0.21282663190076195 0.634
# 0.21085248695856348 0.634
# 0.2091929895648754 0.634
# 0.2040259930808181 0.634
# 0.20830483920311055 0.634
# 0.20110187393197654 0.634
# 0.19921054775576258 0.634
# 0.19614748720352942 0.634
# 0.19579048879561475 0.634
# 0.19344528532212424 0.634
# 0.20045251332451106 0.634
# 0.18825747694914757 0.634
# 0.18576924889914842 0.634
# 0.18411287775658522 0.634
# 0.18302615584253373 0.634
# 0.18454763429228072 0.634
# 0.17626642470559317 0.635
# 0.17469065098247072 0.679
# 0.1719378298327115 0.661
# 0.17605269372506277 0.637
# 0.19701745527854836 0.634
# 0.1608919682268122 0.716
# 0.16342926706599922 0.651
# 0.19137490242816632 0.938
# 0.16993445388782424 0.662
# 0.17335348387678517 0.65
# 0.15162184456043243 0.682
# 0.14235256482453107 0.807
# 0.1512034857104882 0.69
# 0.15666428550872563 0.681
# 0.2567495441627176 0.634
# 0.15486582586192096 0.95
# 0.19636555320933272 0.652
# 0.18016546272626016 0.84
# 0.128967559491965 0.742
# 0.13573770765700863 0.862
# 0.09399810441538305 0.921
# 0.09851463017361554 0.903
# 0.11583907110986205 0.943
# 0.32636879914274497 0.634
# 0.2195528529634276 0.681
# 0.2232071483693024 0.676
# 0.08403512410215158 0.976

In [ ]:
import pickle
X=None
with open('news_sports_train_X.p', 'rb') as f:
    X=pickle.load(f)

X_test=None
with open('news_sports_test_X.p', 'rb') as f:
    X_test=pickle.load(f)

In [ ]:
y=None
with open('news_sports_train_y.p', 'rb') as f:
    y=pickle.load(f)

y_test=None
with open('news_sports_test_y.p', 'rb') as f:
    y_test=pickle.load(f)

In [ ]:
# prepare targets
def get_ts(labels):
    ts = []
    for label in labels:
        l = bool(label)
        ts.append(np.array([not l, l], dtype=bool))
    return ts

In [ ]:
ts = get_ts(y)
ts_test = get_ts(y_test)

In [ ]:
from collections import Counter

In [ ]:
def top_n_words(sentences,n):
    counts = Counter()
    for sen in range(len(sentences)):
        for word in range(len(sentences[sen])):
            counts[sentences[sen][word]] += 1
    return [count[0] for count in counts.most_common(n)]

In [ ]:
top_n_words(X,10)
# [',', '--', 'the', '>', '.', ':', ')', '(', 'to', 'a']

[',', '--', 'the', '>', '.', ':', ')', '(', 'to', 'a']

In [ ]:
def to_hot_encoding(sentences, top_words):
    encoding = []
    for sen in range(len(sentences)):
        sen_encoding = np.zeros(len(top_words), dtype=bool)
        for tw in range(len(top_words)):
            if top_words[tw] in sentences[sen]:
                sen_encoding[tw] = True;
        encoding.append(sen_encoding)
    return encoding


In [ ]:
to_hot_encoding(X[:2], top_n_words(X,10))
#[array([ True,  True,  True,  True,  True,  True,  True,  True,  True, False]),
# array([ True,  True,  True,  True,  True,  True,  True,  True,  True, True])]

[array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        False]),
 array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True])]

In [ ]:
top_words = top_n_words(X, 128)
arrs = to_hot_encoding(X, top_words)

In [ ]:
arrs_test = to_hot_encoding(X_test, top_words)

In [ ]:
my_seed = 2
my_sizes = (128,64,32,2)
my_activations = (relu, tanh, sigmoid) #(relu, tanh, sigmoid)
my_epochs = 17 #32
my_chunks = 50

In [ ]:
# training loop for task g

np.random.seed(my_seed)
myNN = MLP(my_sizes, my_activations)
n_chunks = my_chunks
n_epochs = my_epochs
for i in range(n_epochs):
    for x_chunk, t_chunk in zip(np.array_split(arrs,n_chunks), np.array_split(ts,n_chunks)):
        myNN.update(x_chunk, t_chunk, squared_loss, 0.01)
        # evaluate NN on entire dataset
    print(i)
    print("#", myNN.calc_loss(arrs,ts, squared_loss), myNN.calc_precision(arrs, ts))
    print("# test", myNN.calc_loss(arrs_test,ts_test, squared_loss), myNN.calc_precision(arrs_test, ts_test))

0
# 0.23262962330031403 0.6541274817136886
# test 0.24795578657593675 0.5841708542713567
1
# 0.20212388102198167 0.7053291536050157
# test 0.2239855720401033 0.6369346733668342
2
# 0.16870554407840171 0.7596656217345873
# test 0.20049376097426402 0.6934673366834171
3
# 0.14183293395416852 0.8066875653082549
# test 0.18413570774034732 0.7173366834170855
4
# 0.12625565577064038 0.8286311389759665
# test 0.17738280431613504 0.7298994974874372
5
# 0.10972468779135945 0.8557993730407524
# test 0.1716883095005804 0.742462311557789
6
# 0.11389447971048172 0.8537095088819227
# test 0.17714632810174072 0.7361809045226131
7
# 0.15790282348608978 0.7565308254963428
# test 0.21765937976857913 0.6834170854271356
8
# 0.15780070338777324 0.7617554858934169
# test 0.21914941911865682 0.6909547738693468
9
# 0.1288806053979431 0.8234064785788924
# test 0.19840373616142393 0.7047738693467337
10
# 0.09646304253826962 0.8766980146290491
# test 0.17442066150653404 0.7437185929648241
11
# 0.08202859214080102

# Task 2: Viterbi algorithm
Write a function that implements the Viterbi algorithm using beamsearch. The function takes as input a sentence, state transition probability and word emission probability. State transition and word emission probabilities are provided below.

For each step the beam is expanded with potential pos tags. The most promising examples are then retained.

The function returns the beam after the final word has been processed.
The beam is a list of 2-tuples. The first entry is the probability of that sentence, the second entry is a tuple that contains all the pos-tags that were assigned to the words.

Hint: you may want to use the heapq https://docs.python.org/3/library/heapq.html functions for your beam

In [ ]:
def Viterbi(trans_dict, word_emission_prob, sentence, beam_width):
    res = []
    candidates = []
    trans_keys = get_keys(trans_dict, '<s>')
    prob_keys = get_keys(word_emission_prob, sentence[0])
    for tk in trans_keys:
        pk = (sentence[0], tk[1])
        if (pk in prob_keys):
            candidates.append((trans_dict[tk] * word_emission_prob[pk], [pk[1]]))
    res = sorted(candidates, key=lambda x: (x[0]))[:beam_width]

    for word in range(len(sentence) - 1):
        candidates = []

        prob_keys = get_keys(word_emission_prob, sentence[word+1])
        for path in res:
            trans_keys = get_keys(trans_dict, path[1][len(path[1]) - 1])
            for tk in trans_keys:
                pk = (sentence[word+1], tk[1])
                if (pk in prob_keys):
                    candidates.append((path[0] * trans_dict[tk] * word_emission_prob[pk], path[1] + [tk[1]]))
        res = sorted(candidates, key=lambda x: (-x[0]))[:beam_width]

    return res

def get_keys(dictionary, key):
    return list(filter(lambda k: k[0] == key, dictionary.keys()))

In [ ]:
state_trans_prob = {('<s>','NNP'):0.2767,('<s>','MD'):0.006,('<s>','VB'):0.0031,('<s>','JJ'):0.0453,('<s>','NN'):0.0449,
                   ('<s>','RB'):0.0510,('<s>','DT'):0.2026,
                   ('NNP','NNP'):0.3777,('NNP','MD'):0.0110,('NNP','VB'):0.0009,('NNP','JJ'):0.0084,('NNP','NN'):0.0584,
                   ('NNP','RB'):0.0090,('NNP','DT'):0.0025,
                   ('MD','NNP'):0.0008,('MD','MD'):0.0002,('MD','VB'):0.7968,('MD','JJ'):0.0005,('MD','NN'):0.0008,
                   ('MD','RB'):0.1698,('MD','DT'):0.0041,
                   ('VB','NNP'):0.0322,('VB','MD'):0.0005,('VB','VB'):0.0050,('VB','JJ'):0.0837,('VB','NN'):0.0615,
                   ('VB','RB'):0.0514,('VB','DT'):0.2231,
                   ('JJ','NNP'):0.0366,('JJ','MD'):0.0004,('JJ','VB'):0.0001,('JJ','JJ'):0.0733,('JJ','NN'):0.4509,
                   ('JJ','RB'):0.0036,('JJ','DT'):0.0036,
                   ('NN','NNP'):0.0096,('NN','MD'):0.0176,('NN','VB'):0.0014,('NN','JJ'):0.0086,('NN','NN'):0.1216,
                   ('NN','RB'):0.0177,('NN','DT'):0.0068,
                   ('RB','NNP'):0.0068,('RB','MD'):0.0102,('RB','VB'):0.1011,('RB','JJ'):0.1012,('RB','NN'):0.0120,
                   ('RB','RB'):0.0728,('RB','DT'):0.0479,
                   ('DT','NNP'):0.1147,('DT','MD'):0.0021,('DT','VB'):0.0002,('DT','JJ'):0.2157,('DT','NN'):0.4744,
                   ('DT','RB'):0.0102,('DT','DT'):0.0017}

In [ ]:
word_emission_prob = {('Janet','NNP'):0.000032, ('will','MD'):0.308431,('will','VB'):0.000028,('will','NN'):0.0002,
                     ('back','VB'):0.000672,('back','JJ'):0.00034,('back','NN'):0.000223,('back','RB'):0.010446,
                     ('the','NNP'):0.000048,('the','DT'):0.506099,('bill','VB'):0.000028,('bill','NN'):0.002337}
# assume the missing entries are 0

In [ ]:
sentence = ['Janet', 'will', 'back', 'the', 'bill']

In [ ]:
Viterbi(state_trans_prob, word_emission_prob, sentence, 1)

# [(1.4320953590187012e-15, ('NNP', 'MD', 'RB', 'DT', 'NN'))]

[(1.4320953590187012e-15, ['NNP', 'MD', 'RB', 'DT', 'NN'])]

In [ ]:
Viterbi(state_trans_prob, word_emission_prob, sentence, 2)

# [(2.013570710221386e-15, ('NNP', 'MD', 'VB', 'DT', 'NN')),
# (1.4320953590187012e-15, ('NNP', 'MD', 'RB', 'DT', 'NN'))]

[(2.013570710221386e-15, ['NNP', 'MD', 'VB', 'DT', 'NN']),
 (1.4320953590187012e-15, ['NNP', 'MD', 'RB', 'DT', 'NN'])]

In [ ]:
Viterbi(state_trans_prob, word_emission_prob, sentence, 3)

#[(2.013570710221386e-15, (''NNP', 'MD', 'VB', 'DT', 'NN')),
# (1.4320953590187012e-15, ('NNP', 'MD', 'RB', 'DT', 'NN')),
# (5.139248921926215e-19, ('NNP', 'NN', 'RB', 'DT', 'NN'))]

[(2.013570710221386e-15, ['NNP', 'MD', 'VB', 'DT', 'NN']),
 (1.4320953590187012e-15, ['NNP', 'MD', 'RB', 'DT', 'NN']),
 (5.139248921926215e-19, ['NNP', 'NN', 'RB', 'DT', 'NN'])]